# 2. Building a Model

This documentation workbook describes the method in which we create a model from scratch. First, we create a model which serves as the backbone for all the objects we include within the model. Thereafter, we add metabolites, reactions, setting initial conditions and parameters, running the simulations and plotting said simultions. 


## 2.1 Creating a Model 

For the case of this notebook, we will look as a simple equation: 

$$\begin{equation} x_1 \underset{v_{-1}}{\stackrel{v_1}{\rightleftharpoons}} x_2 \stackrel{v_2} \rightarrow x_3 \end{equation}$$ 


We will consider the reaction linear, irreversible reaction as depicted above. 

In [1]:
from mass import MassModel, MassMetabolite, MassReaction, Simulation, plot_simulation

The above function imports objects into the masspy environment which are required for the creation of a model. First, we will start off by creating aa test type model,

In [2]:
model = MassModel('Toy_Model')
model.description = 'Example Model used to Describe Simple Irreversible 2 Step Reaction'

We can check to see this model rendered in a table

In [3]:
model

Name,Toy_Model
Memory address,0x011e87fd30
Stoichiometric Matrix,0x0
Matrix Rank,0
Matrix Type,"dense, float64"
Number of Metabolites,0
Number of Reactions,0
Number of Initial Conditions,0
Number of Forward Rate Constants,0
Number of Equilibrium Constants,0
Number of Irreversible Reactions,0


As there is no initialization of metabolites or their conditions, we see an empty model; as such, we will incorporate metabolites, reactions, initial conditions and parameters along with running/plotting simulations of our model. 



## 2.2 Mass Metabolites and Reactions

### 2.2.1 Metabolites

We will start with the initialization of the conditions which defines the metabolites and reactions associated with this reaction. 

#### Creating Metabolites 
One method for creating the model is to objects that represent the metabolites and reactions. 
Metabolite are represented by MassMetabolite objects, and can be created by providing a unique identifier for that object. Therefore we can define the four metabolites, \\(x_1, x_2\\) and \\(x_3\\) through the following method:

In [4]:
x1 = MassMetabolite('x1')
x2 = MassMetabolite('x2')
x3 = MassMetabolite('x3')

### 2.2.2 Reactions

#### Creating Reactions

Reactions are represented by MassReactions objects, and like metabolites, they can be also created by providing a unique identifier for that object.

In [5]:
v1= MassReaction('v1')

By default, a reaction is considered reversible. However, if we wish to make an irreversible reaction, we set the *reversible* argument to False. 

In [6]:
v2 = MassReaction('v2', reversible= False)

### 2.2.3 Adding Metabolites to Reactions

After you have introduced both the metabolites, you must add metabolites to the corresponding reactions and then push them into the model. To do this, we add both the initialized metabolite to the reactions which is conversely added to the model. For this, we will use the __MassReaction.add_metabolites__ function to insert metabolites within the model. Therefore, type:

In [7]:
v1.add_metabolites({x1 : -1, x2 : 1})
v2.add_metabolites({x2 : -1, x3 : 1})

### 2.2.4 Adding Reactions to Model

After this, we can add the specific reactions to the model itself:

In [8]:
model.add_reactions([v1,v2])

Let's check to make sure the reactions have all been added to the model, 

In [9]:
# Now there are things in the model
print('%i reaction' % len(model.reactions))
print('%i metabolites' % len(model.metabolites))

2 reaction
3 metabolites


### 2.2.5 Other Ways to Make Models

Apart from the traditional method of creating metabolites & reactions for the model and adding it thereafter, we can also approach model making in non-traditional method 

## 2.3 Setting Parameters and Initial Conditions

Let us check the model again: 

In [10]:
model

Name,Toy_Model
Memory address,0x011e87fd30
Stoichiometric Matrix,3x2
Matrix Rank,2
Matrix Type,"dense, float64"
Number of Metabolites,3
Number of Reactions,2
Number of Initial Conditions,0
Number of Forward Rate Constants,0
Number of Equilibrium Constants,1
Number of Irreversible Reactions,1


From the overview, we realize that no initial conditions or parameters exist; therefore, we will set initial conditions and set parameters that are relavant for the reaction to proceed.

#### Defining Parameters

Parameters can be defined directly by, 

In [11]:
v1.forward_rate_constant = 1
v2.kf = 0.001       #shorthand 

The commands above set forward rate constants for rate v1 and v2; the next step shows how to set equilibrium constants for v1 and v2.

In [12]:
v1.equilibrium_constant = 1 

Naturally, equilibrium constants cannot be set for irreversible reactions; to check the parameters, we can use a "for" loop to iterate for each of the respective parameters, 

In [13]:
for param_type, param_dict in model.parameters.items():
    print ("%s: %s" %(param_type, param_dict))

kf: {'kf_v1': 1, 'kf_v2': 0.001}
Keq: {'Keq_v1': 1, 'Keq_v2': inf}
kr: {}
v: {}
Custom: {}
Fixed: {}


#### Defining Initial Conditions

Initial Conditions for each metabolite can be defined directly and added to the metabolites itself as they are attributes of the object, 

In [14]:
x1.initial_condition = 1
x2.ic = 0
model.set_initial_conditions()
model.initial_conditions

{<MassMetabolite x1 at 0x11e8fd048>: 1, <MassMetabolite x2 at 0x11e8fd1d0>: 0}

Of course, conditions for a model are not set; they can be updated within the object which is the metabolite and will be *extremely useful when perturbing the system/model*. To update the initial conditions, type

In [15]:
model.update_initial_conditions({x3:0}, update_metabolites=True)
model.initial_conditions

{<MassMetabolite x1 at 0x11e8fd048>: 1,
 <MassMetabolite x2 at 0x11e8fd1d0>: 0,
 <MassMetabolite x3 at 0x11e8fd080>: 0}

With the initial conditions set, we have successfully created a cohesive model which can be perturbed and simulated to the best manner possible. 

## 2.4 Creating complex models

Now, let's try and create more complex models and see how the creation of these models would be accomplished in a normal workflow. Consider the reacion to be,

$$\begin{equation} x_1 + x_2 \underset{v_{-1}}{\stackrel{v_1}{\rightleftharpoons}} x_3 \stackrel{v_2} \rightarrow x_4 \underset{v_{-3}}{\stackrel{v_3}{\rightleftharpoons}} x_5 \end{equation}$$ 

This is a more complicated model; a connected bi-linear irreversible equation. However, the same principle applies here as well; for the sake of brevity, we will not highlight each and every step but display how this would be done in an actual model. 

In [16]:
# Create MassModel
model = MassModel('Complex Model')
# Generate the MassMetabolites 
x1 = MassMetabolite("x1")
x2 = MassMetabolite("x2")
x3 = MassMetabolite("x3")
x4 = MassMetabolite("x4")
x5 = MassMetabolite("x5")
# Generate the MassReactions 
v1 = MassReaction("v1")
v2 = MassReaction("v2", reversible=False)
v3 = MassReaction("v3")
# Add metabolites to the reaction, add reaction to the model
v1.add_metabolites({x1: -1, x2: -1, x3: 1})
v2.add_metabolites({x3: -1, x4: 1})
v3.add_metabolites({x4: -1, x5: 1})
model.add_reactions([v1, v2, v3])
# Set parameters and initial conditions
v1.kf = 1
v1.Keq = 1
v2.kf = 1
v3.kf = 1
v3.Keq = 1
model.update_initial_conditions({x1: 1, x2: 1, x3: 0, x4: 0, x5:0 }, 
                                update_metabolites=True)

Let's pull up the model we created to see how it looks like, 

In [17]:
model

Name,Complex Model
Memory address,0x011e92eb00
Stoichiometric Matrix,5x3
Matrix Rank,3
Matrix Type,"dense, float64"
Number of Metabolites,5
Number of Reactions,3
Number of Initial Conditions,5
Number of Forward Rate Constants,3
Number of Equilibrium Constants,3
Number of Irreversible Reactions,1


The model above is just an example of many more complex reactions you will create throughout the course of using MASSPy; getting familiar with the initialization and usage of model creations. 

## 2.5 Open Systems

In the previous examples, we only considered closed systems which typically demonstrate reactions in environments where influences outside the system were not simulated. However, living systems are defined as having matter and energy flow into and out of the system which pushes them away from equilibrium. Therefore, we will now discuss how to create an open system which has an influx and outflux of matter and molecules.

Using out toy model, we will add an inflow \\((b_1)\\) of \\(x_1\\) and an outflow \\((b_2)\\) of \\(x_3\\):

$$\begin{equation} \stackrel{b_1}{\rightarrow} x_1 \underset{v_{-1}}{\stackrel{v_1}{\rightleftharpoons}} x_2 \stackrel{v_2} \rightarrow x_3 \stackrel{b_2}{\rightarrow} \end{equation}$$

This is a good time to talk about updating metabolites; since we are using a previous example, there may be many instances where you want to update your model without re-writing your entire code. To do so, we can update reactions, metabolites and other aspects of the models so as to make perturbations easier, 
<font color='green'>I think it would be nice to have the update function displayed here and for that I think I need this model to be made into something I can pull up more quickly? </font>

In [20]:
# Create MassModel
model = MassModel('Toy_Model')
# Generate the MassMetabolites 
x1 = MassMetabolite("x1")
x2 = MassMetabolite("x2")
x3 = MassMetabolite("x3")
# Generate the MassReactions 
v1 = MassReaction("v1")
v2 = MassReaction("v2", reversible=False)

One distinction between open and closed systems is the inclusion of the inflow and outflow as MassReaction objects,

In [22]:
b1 = MassReaction("b1",reversible=False)
b2 = MassReaction("b2",reversible=False)

In [25]:
# Add metabolites to the reaction, add reaction to the model
b1.add_metabolites({x1: 1})
v1.add_metabolites({x1: -1, x2: 1})
v2.add_metabolites({x2: -1, x3: 1})
b2.add_metabolites({x3: -1})
model.add_reactions([b1, v1, v2, b2])
#Set Parameters & Initial Conditions
b1.kf = 0.001
v1.kf, v1.kr = (1, 2)
v2.kf = 0.1
b2.kf = 0.01
model.set_initial_conditions()
# Set a custom rate for b1 to remove substrate concentration dependence
model.add_custom_rate(reaction=b1, custom_rate=b1.kf_str)

Ignoring reaction 'b1' since it already exists.
Ignoring reaction 'v1' since it already exists.
Ignoring reaction 'v2' since it already exists.
Ignoring reaction 'b2' since it already exists.


From this, we have created an open system which contains fixed boundaries and initial conditions along with an influx that is set to a custom rate. 
<font color='green'>why do we set a custom rate? </font>

In [26]:
model

Name,Toy_Model
Memory address,0x011e924f28
Stoichiometric Matrix,3x4
Matrix Rank,3
Matrix Type,"dense, float64"
Number of Metabolites,3
Number of Reactions,4
Number of Initial Conditions,0
Number of Forward Rate Constants,4
Number of Equilibrium Constants,3
Number of Irreversible Reactions,3


Open Systems are much more realistic and are used much more frequently when creating models. The toy model served as a method to learn efficiently how to make an open model; with very little difference apart from the adition of 2 more irreversible reactions, we can observe how easy it is to create such a model.